In [212]:
import random
import math

random.seed(0)
conv = 'Convolution'
fc   = 'FullConnect'
conv3_3  = 9
conv1_1  = 1
timescale = 200 * 1000

def NetCal(Net):
    img_size    = Net['img_size']
    conv_size   = Net['conv_size']
    ch_in       = Net['ch_in']
    ch_out      = Net['ch_out']
    bias_len    = Net['ch_out']
    shortcut    = Net['shortcut']
    type        = Net['type']
    img_h       = Net['img_size']
    img_w       = Net['img_size']

    #理论数据长度
    img_len  = img_size * img_size * ch_in
    wei_len  = ch_in    * ch_out   * conv_size

    #理论吞吐量与计算量
    io_amount  = (img_len + wei_len + bias_len*2)/1000000
    cal_amount = (ch_out* ch_in * img_size * img_size * conv_size)/1000000

    #实际数据长度
    if(Net['img_size']%1):
        Net['img_size'] = Net['img_size'] + 1

    x=math.log(ch_out,2)
    if not x.is_integer():
        ch_out = pow(2,math.ceil(x))

    if ch_in%8 > 0:
        ch_in = 8 * math.ceil(ch_in/8)

    # Calcount_real = img_size * img_size * ch_in * conv_size
    # img_len_real  = img_size * img_size * ch_in
    wei_len_real  = ch_in    * ch_out   * conv_size
    bias_len_real = ch_out

    #理论上硬件运行时间
    iotime  = (wei_len_real/16 + bias_len_real/8) / timescale
    caltime = ch_out/32 * ch_in/8 * img_size/2 * (img_size+1) / timescale
    totaltime = iotime + caltime

    if type == conv :
        dsptime = caltime
    else :
        dsptime = ch_out/32 * ch_in/8 * img_size/2 * img_size / timescale

    #实际运行时间
    totaltime_real = round(totaltime * (1.01 + shortcut*0.01 + random.uniform(0.1,1)/100),2)
    dspusage_real  = round(dsptime/totaltime * conv_size/9,4) #百分数

    res = {'type':type,'size':str(round(pow(conv_size,0.5)))+'x'+str(round(pow(conv_size,0.5))),'para':str(img_size)+'x'+str(img_size)+'x'+str(ch_in)+'x'+str(ch_out),
            'cal_amount':cal_amount,'io_amount':io_amount, 'io_time':iotime, 
            'totaltime_real':totaltime_real, 'dspusage_real':dspusage_real}

    print(res)
    return(res)


def TotalCal(datas):
    total_cal_amount = 0
    total_io_amount  = 0
    total_time       = 0
    total_dspusage   = 0
    total_conv_time  = 0
    total_fc_time    = 0
    total_io_time    = 0

    dsp_temp         = 0

    for data in datas :
        res = NetCal(data)
        total_cal_amount = total_cal_amount + res['cal_amount']
        total_io_amount  = total_io_amount + res['io_amount']
        total_time       = total_time + res['totaltime_real']
        dsp_temp         = dsp_temp + res['totaltime_real'] * res['dspusage_real']
        total_io_time    = total_io_time + res['io_time']

        if(res['type']==conv):
            total_conv_time = total_conv_time + res['totaltime_real']
        else:
            total_fc_time = total_fc_time + res['totaltime_real']
        

    total_dspusage = dsp_temp / total_time

    res = {'type':'Total','total_cal_amount':round(total_cal_amount,2),'total_io_amount':round(total_io_amount,2), 
            'total_dspusage':round(total_dspusage*100,2),'total_time':round(total_time,2),'total_io_time':round(total_io_time,2),
            'total_conv_time':round(total_conv_time,2),'total_fc_time':round(total_fc_time,2)
            }
    # print(res)
    
    return(res)


In [213]:
# VGG 16 计算

VGG16=[]
VGG16.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 224, 'ch_in': 3    , 'ch_out'  : 64  , 'shortcut': 0})
VGG16.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 224, 'ch_in': 64   , 'ch_out'  : 64  , 'shortcut': 0})
VGG16.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 112, 'ch_in': 64   , 'ch_out'  : 128 , 'shortcut': 0})
VGG16.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 112, 'ch_in': 128  , 'ch_out'  : 128 , 'shortcut': 0})
VGG16.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 56 , 'ch_in': 128  , 'ch_out'  : 256 , 'shortcut': 0})
VGG16.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 56 , 'ch_in': 256  , 'ch_out'  : 256 , 'shortcut': 0})
VGG16.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 56 , 'ch_in': 256  , 'ch_out'  : 256 , 'shortcut': 0})
VGG16.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 28 , 'ch_in': 256  , 'ch_out'  : 512 , 'shortcut': 0})
VGG16.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 28 , 'ch_in': 512  , 'ch_out'  : 512 , 'shortcut': 0})
VGG16.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 28 , 'ch_in': 512  , 'ch_out'  : 512 , 'shortcut': 0})
VGG16.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 14 , 'ch_in': 512  , 'ch_out'  : 512 , 'shortcut': 0})
VGG16.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 14 , 'ch_in': 512  , 'ch_out'  : 512 , 'shortcut': 0})
VGG16.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 14 , 'ch_in': 512  , 'ch_out'  : 512 , 'shortcut': 0})
VGG16.append({ 'type': fc  , 'conv_size':conv1_1, 'img_size': 1  , 'ch_in': 25088, 'ch_out'  : 4096, 'shortcut': 0})
VGG16.append({ 'type': fc  , 'conv_size':conv1_1, 'img_size': 1  , 'ch_in': 4096 , 'ch_out'  : 4096, 'shortcut': 0})
VGG16.append({ 'type': fc  , 'conv_size':conv1_1, 'img_size': 1  , 'ch_in': 4096 , 'ch_out'  : 1000, 'shortcut': 0})

# NetCal(VGG16[-1])
TotalCal(VGG16)


{'type': 'Convolution', 'size': '3x3', 'para': '224x224x8x64', 'cal_amount': 86.704128, 'io_amount': 0.152384, 'io_time': 0.00148, 'totaltime_real': 0.26, 'dspusage_real': 0.9942}
{'type': 'Convolution', 'size': '3x3', 'para': '224x224x64x64', 'cal_amount': 1849.688064, 'io_amount': 3.248256, 'io_time': 0.01156, 'totaltime_real': 2.06, 'dspusage_real': 0.9943}
{'type': 'Convolution', 'size': '3x3', 'para': '112x112x64x128', 'cal_amount': 924.844032, 'io_amount': 0.8768, 'io_time': 0.02312, 'totaltime_real': 1.05, 'dspusage_real': 0.9777}
{'type': 'Convolution', 'size': '3x3', 'para': '112x112x128x128', 'cal_amount': 1849.688064, 'io_amount': 1.753344, 'io_time': 0.04616, 'totaltime_real': 2.1, 'dspusage_real': 0.9777}
{'type': 'Convolution', 'size': '3x3', 'para': '56x56x128x256', 'cal_amount': 924.844032, 'io_amount': 0.696832, 'io_time': 0.09232, 'totaltime_real': 1.13, 'dspusage_real': 0.9171}
{'type': 'Convolution', 'size': '3x3', 'para': '56x56x256x256', 'cal_amount': 1849.688064,

{'type': 'Total',
 'total_cal_amount': 15470.26,
 'total_io_amount': 147.49,
 'total_dspusage': 27.56,
 'total_time': 63.98,
 'total_io_time': 43.27,
 'total_conv_time': 22.17,
 'total_fc_time': 41.81}

In [214]:
# Resnet 18
Resnet18=[]
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 224, 'ch_in': 3    , 'ch_out'  : 64  , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 112, 'ch_in': 64   , 'ch_out'  : 64  , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 112, 'ch_in': 64   , 'ch_out'  : 64  , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 112, 'ch_in': 64   , 'ch_out'  : 64  , 'shortcut': 0})

Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 56 , 'ch_in': 64   , 'ch_out'  : 64  , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 56 , 'ch_in': 64   , 'ch_out'  : 64  , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 56 , 'ch_in': 64   , 'ch_out'  : 64  , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 56 , 'ch_in': 64   , 'ch_out'  : 64  , 'shortcut': 0})

Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 28 , 'ch_in': 64   , 'ch_out'  : 128 , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 28 , 'ch_in': 128  , 'ch_out'  : 128 , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 28 , 'ch_in': 128  , 'ch_out'  : 128 , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 28 , 'ch_in': 128  , 'ch_out'  : 128 , 'shortcut': 0})

Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 14 , 'ch_in': 128  , 'ch_out'  : 256 , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 14 , 'ch_in': 256  , 'ch_out'  : 256 , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 14 , 'ch_in': 256  , 'ch_out'  : 256 , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 14 , 'ch_in': 256  , 'ch_out'  : 512 , 'shortcut': 0})

Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 7  , 'ch_in': 512  , 'ch_out'  : 512 , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 7  , 'ch_in': 512  , 'ch_out'  : 512 , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 7  , 'ch_in': 512  , 'ch_out'  : 512 , 'shortcut': 0})
Resnet18.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 7  , 'ch_in': 512  , 'ch_out'  : 512 , 'shortcut': 0})

Resnet18.append({ 'type': fc  , 'conv_size':conv1_1, 'img_size': 1  , 'ch_in': 25088, 'ch_out'  : 1000, 'shortcut': 0})

TotalCal(Resnet18)

{'type': 'Convolution', 'size': '3x3', 'para': '224x224x8x64', 'cal_amount': 86.704128, 'io_amount': 0.152384, 'io_time': 0.00148, 'totaltime_real': 0.26, 'dspusage_real': 0.9942}
{'type': 'Convolution', 'size': '3x3', 'para': '112x112x64x64', 'cal_amount': 462.422016, 'io_amount': 0.839808, 'io_time': 0.01156, 'totaltime_real': 0.53, 'dspusage_real': 0.9777}
{'type': 'Convolution', 'size': '3x3', 'para': '112x112x64x64', 'cal_amount': 462.422016, 'io_amount': 0.839808, 'io_time': 0.01156, 'totaltime_real': 0.53, 'dspusage_real': 0.9777}
{'type': 'Convolution', 'size': '3x3', 'para': '112x112x64x64', 'cal_amount': 462.422016, 'io_amount': 0.839808, 'io_time': 0.01156, 'totaltime_real': 0.53, 'dspusage_real': 0.9777}
{'type': 'Convolution', 'size': '3x3', 'para': '56x56x64x64', 'cal_amount': 115.605504, 'io_amount': 0.237696, 'io_time': 0.01156, 'totaltime_real': 0.14, 'dspusage_real': 0.917}
{'type': 'Convolution', 'size': '3x3', 'para': '56x56x64x64', 'cal_amount': 115.605504, 'io_amo

{'type': 'Total',
 'total_cal_amount': 3348.75,
 'total_io_amount': 41.98,
 'total_dspusage': 23.97,
 'total_time': 16.76,
 'total_io_time': 12.05,
 'total_conv_time': 8.08,
 'total_fc_time': 8.68}

In [215]:
# Darknet 19
Darknet19=[]
Darknet19.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 224, 'ch_in': 3    , 'ch_out'  : 32  , 'shortcut': 0})
Darknet19.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 112, 'ch_in': 32   , 'ch_out'  : 64  , 'shortcut': 0})

Darknet19.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 56 , 'ch_in': 64   , 'ch_out'  : 128 , 'shortcut': 0})
Darknet19.append({ 'type': conv, 'conv_size':conv1_1, 'img_size': 56 , 'ch_in': 128  , 'ch_out'  : 64  , 'shortcut': 0})
Darknet19.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 56 , 'ch_in': 64   , 'ch_out'  : 128 , 'shortcut': 0})

Darknet19.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 28 , 'ch_in': 128  , 'ch_out'  : 256 , 'shortcut': 0})
Darknet19.append({ 'type': conv, 'conv_size':conv1_1, 'img_size': 28 , 'ch_in': 256  , 'ch_out'  : 128 , 'shortcut': 0})
Darknet19.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 28 , 'ch_in': 128  , 'ch_out'  : 256 , 'shortcut': 0})

Darknet19.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 14 , 'ch_in': 256  , 'ch_out'  : 512 , 'shortcut': 0})
Darknet19.append({ 'type': conv, 'conv_size':conv1_1, 'img_size': 14 , 'ch_in': 512  , 'ch_out'  : 256 , 'shortcut': 0})
Darknet19.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 14 , 'ch_in': 256  , 'ch_out'  : 512 , 'shortcut': 0})
Darknet19.append({ 'type': conv, 'conv_size':conv1_1, 'img_size': 14 , 'ch_in': 512  , 'ch_out'  : 256 , 'shortcut': 0})
Darknet19.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 14 , 'ch_in': 256  , 'ch_out'  : 512 , 'shortcut': 0})

Darknet19.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 7  , 'ch_in': 512  , 'ch_out'  : 1024, 'shortcut': 0})
Darknet19.append({ 'type': conv, 'conv_size':conv1_1, 'img_size': 7  , 'ch_in': 1024 , 'ch_out'  : 512 , 'shortcut': 0})
Darknet19.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 7  , 'ch_in': 512  , 'ch_out'  : 1024, 'shortcut': 0})
Darknet19.append({ 'type': conv, 'conv_size':conv1_1, 'img_size': 7  , 'ch_in': 1024 , 'ch_out'  : 512 , 'shortcut': 0})
Darknet19.append({ 'type': conv, 'conv_size':conv3_3, 'img_size': 7  , 'ch_in': 1024 , 'ch_out'  : 512 , 'shortcut': 0})
Darknet19.append({ 'type': conv, 'conv_size':conv1_1, 'img_size': 7  , 'ch_in': 512  , 'ch_out'  : 1000, 'shortcut': 0})

TotalCal(Darknet19)

{'type': 'Convolution', 'size': '3x3', 'para': '224x224x8x32', 'cal_amount': 43.352064, 'io_amount': 0.151456, 'io_time': 0.00074, 'totaltime_real': 0.13, 'dspusage_real': 0.9942}
{'type': 'Convolution', 'size': '3x3', 'para': '112x112x32x64', 'cal_amount': 231.211008, 'io_amount': 0.419968, 'io_time': 0.0058, 'totaltime_real': 0.26, 'dspusage_real': 0.9776}
{'type': 'Convolution', 'size': '3x3', 'para': '56x56x64x128', 'cal_amount': 231.211008, 'io_amount': 0.274688, 'io_time': 0.02312, 'totaltime_real': 0.28, 'dspusage_real': 0.917}
{'type': 'Convolution', 'size': '1x1', 'para': '56x56x128x64', 'cal_amount': 25.690112, 'io_amount': 0.409728, 'io_time': 0.0026, 'totaltime_real': 0.26, 'dspusage_real': 0.11}
{'type': 'Convolution', 'size': '3x3', 'para': '56x56x64x128', 'cal_amount': 231.211008, 'io_amount': 0.274688, 'io_time': 0.02312, 'totaltime_real': 0.28, 'dspusage_real': 0.917}
{'type': 'Convolution', 'size': '3x3', 'para': '28x28x128x256', 'cal_amount': 231.211008, 'io_amount':

{'type': 'Total',
 'total_cal_amount': 2765.9,
 'total_io_amount': 22.66,
 'total_dspusage': 28.99,
 'total_time': 11.51,
 'total_io_time': 6.36,
 'total_conv_time': 11.51,
 'total_fc_time': 0}